In [ ]:
# https://github.com/aigeek0x0/zephyr-7b-alpha-langchain-chatbot/tree/main

#All the installations

In [ ]:
# # install required packages
 !pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit weaviate-client langchain sentence-transformers tiktoken youtube-transcript-api

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 50

In this notebook we will see how utilizing RAG will benifit the application as the information provided by us is vectorized and stored in the vector database. Upon the user query the relevant information is then passed as a context along with thw user query for text generation. This helps in avoiding the hallucination from the Langauge model.

In [ ]:
# fixing unicode error in google colab
import locale

locale.getpreferredencoding = lambda: "UTF-8"
import torch
# import dependencies

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.text_splitter import TokenTextSplitter,RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate

In [ ]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
! pip install transformers accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained("anakin87/zephyr-7b-alpha-sharded", device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("anakin87/zephyr-7b-alpha-sharded")


Lets see what the LM generates for some of our queries before RAG

The questions are based on the telugu language , lets see if our model has answers

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a rapper",
    },
    {"role": "user", "content": "What is onamalu in telugu?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>
You are a friendly chatbot who always responds in the style of a rapper</s>
<|user|>
What is onamalu in telugu?</s>
<|assistant|>
In Telugu, "Onamalu" translates to "ఓంమలు" or "ఓం మలు". "Om" is a sacred sound and mantra in Hinduism and Buddhism, and "malulu" translates to "flowers" in Telugu. So, "Onamalu" refers to the flower arrangements or decorations made during the Onam festival in Kerala.


The above answer is wrong as onamalu in telugu is alphabets. Lets try some more questions

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a assistant",
    },
    {"role": "user", "content": "Who is father of modern telugu literature?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>
You are a friendly chatbot who always responds in the style of a assistant</s>
<|user|>
Who is father of modern telugu literature?</s>
<|assistant|>
The father of modern Telugu literature is Gurajada Apparao, also known as Gurajada Vinukonda Rao. He was a renowned Telugu poet, playwright, and social reformer who lived from 1864 to 1928. He is credited with introducing new themes, styles, and techniques to Telugu literature, and his works continue to be popular and influential today.


The answer from the model is not very much true but he is father of telugu poetry not literature, its certain that the model is hallucinating. Lets try one more question

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a assistant",
    },
    {"role": "user", "content": "How did telugu got its name?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>
You are a friendly chatbot who always responds in the style of a assistant</s>
<|user|>
How did telugu got its name?</s>
<|assistant|>
Telugu is a Dravidian language spoken mainly in the Indian states of Andhra Pradesh, Telangana, and Karnataka. The name "Telugu" is derived from the Sanskrit word "Telugu," which means "southern country" or "region." The language is believed to have originated in the southern region of India and has been in use for over a thousand years. The name "Telugu" was first recorded in the 11th century in the works of the Sanskrit scholar Kalhana. Today, Telugu is one of the 22 official languages of India and is spoken by over 80 million people worldwide.


The above answer is not very much wrong , but lets see if llm with RAG works or not

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a assistant",
    },
    {"role": "user", "content": "What is tenugu?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>
You are a friendly chatbot who always responds in the style of a assistant</s>
<|user|>
What is tenugu?</s>
<|assistant|>
Tenugu is a language spoken primarily in the Indian state of Andhra Pradesh, particularly in the Rayalaseema region. The language is also spoken in some parts of Telangana and Karnataka. It is classified as a Dravidian language and is closely related to Telugu. The name "Tenugu" refers to the region where the language originated, and it is derived from the Sanskrit word "Tenakam," meaning "forest land."


Setting pipeline for the text-generation with the above LLM

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:

DB_FAISS_PATH = 'vectorstore/db_faiss'

# Create vector database

loader = TextLoader('/content/telugu_RAG',encoding = 'UTF-8')
documents = loader.load()
print("documents loaded successfully")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                                chunk_overlap=20)
texts = text_splitter.split_documents(documents)
print("text splitting done successfully")




# db = FAISS.from_documents(texts, embeddings)
db= Chroma.from_documents(texts, embeddings,persist_directory=DB_FAISS_PATH)
# pdb.set_trace()
# db.save_local(DB_FAISS_PATH)
print("databases saved successfully")



In [ ]:
g = db.get()


In [ ]:
template = f"""Use the following Context as information to you to answer the user's question.
If you don't know the answer then you are advised to look for the context and answer the question, even if the context does not have answer just say that you don't know , don't try to make up an answer.


Context: {{context}}
Question: {{question}}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1}),
                                            #retriever=db.as_retriever(search_type="mmr"),
                                           return_source_documents=True,
                                            chain_type_kwargs={'prompt': prompt},
                                           )

In [ ]:
response = qa_chain({'query': 'what is onamalu in telugu?'})

In [ ]:
response['result']

'In Telugu, the word "Onamaalu" is used to refer to the Telugu alphabet. It can be translated as "first letter" or "first sound." The word is derived from the combination of two Telugu words: "onu" meaning "first" and "maalu" meaning "letter" or "sound." So, Onamaalu literally means "first letter" or "first sound".'

The above answer is perfect and thus we can see having RAG helps certainly. compare this answer to the one above

In [ ]:
response = qa_chain({'query': 'who is father of modern telugu literature?'})

In [ ]:
response['result']

'\nThe father of modern Telugu literature is Kandukuri Viresalingam Pantulu (1848--1919).'

The above answer is perfect and it is stored in our documents

In [ ]:
response = qa_chain({'query': 'How telugu got its name?'})

In [ ]:
response['result']

'The Telugu Language became known as Telugu through the process of standardized grammar and script development. It is believed to have originated from the Proto-Telugu language spoken in the Andhra Pradesh region of India around the 10th century AD.'

The above answer is not perfect, as the information is not in the documents.

In [ ]:
response = qa_chain({'query': 'what is tenugu?'})

In [ ]:
response['result']

'Tenugu is a region in India that was historically used to describe those traveling towards the south. In Tamil and Kannada, "ten" means ten, and the suffix "gu" means place of, or region of, so Tenugu would translate to "the place of going south by 10" in those languages. However, over time the name has evolved and is now commonly used to refer to the city Telangana, which was historically occupied and renamed.'

The above text is correct compared to the answer with out RAG

So as we have briefly evaluate the LM we can say that having RAG does help in retreiving the relevant information. RAG does help in giving the LM context as finetuning LM's are a bit costly. This is a simple example, we can add memory buffer and create a chatbot for our usecase and also we can scale RAG's using AIRFLOW etc where we feed new data every month or so to our DB. We can make use of chrom db functions like upsert etc to make sure we donot embed similar information.